# Demo of `secop-ophyd` Integration 

This demo utilizes the samplechanger SEC node, and demonstrates the concurrent control and readout of EPICS Detectors and SECoP hardware.

prerequisites:
- a running instance ofURsim (see `/URsimDocker` directory) or the real UR3 robot samplechanger apparatus

- `remote mode`of the robot has to be enabled 
    
- a running SEC node for the samplechanger robot is needed as well:
    ```python3 frappy/bin/frappy-server -c frappy/cfg/UR_robot_cfg.py``` 


### Bluesky Runenegine Setup

In [1]:
from databroker.v2 import temp
from bluesky import RunEngine, Msg
import bluesky.plan_stubs as bps

from pprint import pprint
from bluesky.plans import scan
import bluesky.preprocessors as bpp

from secop_ophyd.SECoPDevices import SECoP_Node_Device, SECoPReadableDevice, SECoPMoveableDevice, SECoP_CMD_Device

import random

from ophyd.sim import  SynGauss, motor

noisy_det0 = SynGauss(
        "noisy_det0",
        motor,
        "motor",
        center=1,
        Imax=1,
        noise="uniform",
        sigma=1,
        noise_multiplier=0.4,
        labels={"detectors"},
    )

noisy_det1 = SynGauss(
        "noisy_det1",
        motor,
        "motor",
        center=0.5,
        Imax=1,
        noise="uniform",
        sigma=1,
        noise_multiplier=1,
        labels={"detectors"},
    )

noisy_det2 = SynGauss(
        "noisy_det2",
        motor,
        "motor",
        center=0,
        Imax=1,
        noise="uniform",
        sigma=1,
        noise_multiplier=3,
        labels={"detectors"},
    )



url_robo = '192.168.15.6'
url_nico = '192.168.15.4'
port_nico = '2201'

# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({})
db = temp()
RE.subscribe(db.v1.insert)

RE.verbose = False



#Example of adding metadata to RE environment
investigation_id = "kmnk2n34"

RE.md["investigation_id"] = investigation_id



Extend DSO search path to '/home/qfj/git-repos/secop-ophyd/.venv/lib/python3.10/site-packages/epicscorelibs/lib'
first instance of OphydObject: id=139826682781200
put(value=0, timestamp=None, force=False, metadata=None)
put(value=0, timestamp=None, force=False, metadata=None)
put(value=1, timestamp=None, force=False, metadata=None)
put(value=1, timestamp=None, force=False, metadata=None)
put(value=2, timestamp=None, force=False, metadata=None)
put(value=0.1, timestamp=None, force=False, metadata=None)
put(value=0.9631770513095513, timestamp=None, force=False, metadata=None)
put(value=0, timestamp=None, force=False, metadata=None)
put(value=0, timestamp=None, force=False, metadata=None)
put(value=1, timestamp=None, force=False, metadata=None)
put(value=1, timestamp=None, force=False, metadata=None)
put(value=1.0, timestamp=None, force=False, metadata=None)
put(value=0, timestamp=None, force=False, metadata=None)
put(value=0, timestamp=None, force=False, metadata=None)
put(value=1, times

### SECoP-Ophyd Device generation

In [2]:


# Connect to SEC Node and generate ophyd device tree
robo_dev = await SECoP_Node_Device.create(url_robo,'10770',RE.loop)


#One Device for every Robot SECoP Module 
storage:SECoPReadableDevice= robo_dev.storage
sample:SECoPMoveableDevice= robo_dev.sample
robot:SECoPMoveableDevice= robo_dev.robot

#Devices for SECoP Commands
loadshort:SECoP_CMD_Device = storage.load_short_CMD
measure:SECoP_CMD_Device = sample.measure_CMD


sample_changer.HZB ready


### Populating the Storage Module

Samples of random substance are loaded into the storage module  

In [3]:



SCHOKO_SORTEN = {
    0:'Knusperkeks',
    1:'Edel-Vollmilch',
    2:'Knusperflakes',
    3:'Nuss-Splitter',
    4:'Nugat',
    5:'Marzipan',
    6:'Joghurt',
}



# Example Plan for loading samples into the robot
def load_samples(storage, lower, upper):
    loadshort:SECoP_CMD_Device = storage.load_short_CMD

    for samplepos in range(lower,upper):
        
        ### populate Command arguments
        
        # set substance of sample
        substance = random.randint(0,6)
        yield from bps.abs_set(loadshort.argument,{'substance':substance,'samplepos':samplepos}, group='sample')         


        ## wait for arguments to be set (should be instant because command args are locally saved dummy signals)
        yield from bps.wait('sample')
        

        # Trigger command execution of _load_short command
        yield from bps.trigger(loadshort,wait=True)        

        print( 'A \'' + SCHOKO_SORTEN[substance] + '\' sample has been loaded into position '+ str(samplepos) + '.')



## loading random samples
RE(load_samples(storage,1,4))
        

Executing plan <generator object load_samples at 0x7f2f925123b0>
Change state on <bluesky.run_engine.RunEngine object at 0x7f2fd17963e0> from 'idle' -> 'running'
set(<ophyd_async.core.signal.SignalRW object at 0x7f2f924f83d0>, *({'substance': 0, 'samplepos': 1},) **{'group': 'sample'}, run=None)
wait(None, *() **{'group': 'sample', 'timeout': None}, run=None)
The object <ophyd_async.core.signal.SignalRW object at 0x7f2f924f83d0> reports set is done with status True
trigger(<secop_ophyd.SECoPDevices.SECoP_CMD_Device object at 0x7f2f924f83a0>, *() **{'group': None}, run=None)
wait(None, *() **{'group': None}, run=None)
The object <secop_ophyd.SECoPDevices.SECoP_CMD_Device object at 0x7f2f924f83a0> reports 'trigger' is done with status True.
A 'Knusperkeks' sample has been loaded into position 1.
set(<ophyd_async.core.signal.SignalRW object at 0x7f2f924f83d0>, *({'substance': 0, 'samplepos': 2},) **{'group': 'sample'}, run=None)
wait(None, *() **{'group': 'sample', 'timeout': None}, run=N

()

### Making a Measurement (`_measure()` Command)

Here a simulated EPICS detector is read while the SECoP `_measure()` Command is running. 

In [ ]:
def measure(sample,sample_num):

    ## Sample Check
    # check if robot is currently holding a sample and 
    # put it back into storage if its not th correct one    
    reading = yield from bps.read(sample)
  
    curr_sample = reading[sample.value.name]['value']
    
    # holding wrong sample -->  put it back into storage
    if curr_sample != 0 and curr_sample  != sample_num :
        yield from bps.mv(sample,0)
    
    # gripper empty --> grab correct sample
    if curr_sample == 0:
        print('grabbing sample: '+str(i))
        yield from bps.mv(sample,i)
    
    print('holding sample: '+str(i))
    
    @bpp.run_decorator()
    def inner_meas(sample):
        
          
        complete_status = yield from bps.trigger(sample.measure_CMD, wait=False) #This message doesn't exist yet
        print('starting measurement')
        
        # While the device is still executing, read from the detectors in the detectors list
        while not complete_status.done:

            yield Msg('checkpoint') # allows us to pause the run 
            
            yield from bps.one_shot([noisy_det0,noisy_det1,noisy_det2]) #triggers and reads everything in the detectors list
           
           
            yield Msg('sleep', None, 1)       
        
        print('measurement done')
            
      
    ### Do actual measurement ###
    uid = yield from inner_meas(sample)

 
    # put sample back into storage
    print('putting back sample: '+str(i))
    yield from  bps.mv(sample,0)

    return uid




for i in range(1,2):
    #grab sample i and hold in Measurement Pos
    RE(measure(sample,i))
    


### Plotting Detector data for the most recent Sample

In [ ]:

run=db[-1]
data = run.primary.read()
data["noisy_det0"].plot()
data["noisy_det1"].plot()
data["noisy_det2"].plot()


# Nico SEC Node


In [2]:


# Connect to SEC Node and generate ophyd device tree


node = await SECoP_Node_Device.create(url_nico,port_nico,RE.loop)


bhm0:SECoPMoveableDevice= node.SECoP_BH_M_0
bhm1:SECoPMoveableDevice= node.SECoP_BH_M_1

print('Property values:')
pprint(await node.read_configuration())
print('Property metadata:')
pprint(await node.describe_configuration())



Node_ID ready
Property values:
{'Node_ID-description': {'timestamp': 1705662281.3229902,
                         'value': 'This is the node for MFC an PC (Bronkhorst '
                                  'GmbH, Germany) where n devices (defined in '
                                  'hardware modules.cfg) call n modules.'},
 'Node_ID-equipment_id': {'timestamp': 1705662281.3229902, 'value': 'Node_ID'},
 'Node_ID-firmware': {'timestamp': 1705662281.3229902,
                      'value': 'SHALL server library (SVN378)'},
 'Node_ID-order': {'timestamp': 

1705662281.3229902,
                   'value': ['SECoP_BH_M_0', 'SECoP_BH_M_1']}}
Property metadata:
{'Node_ID-description': {'dtype': 'string',
                         'shape': [],
                         'source': 'description'},
 'Node_ID-equipment_id': {'dtype': 'string',
                          'shape': [],
                          'source': 'equipment_id'},
 'Node_ID-firmware': {'dtype': 'string', 'shape': [], 'source': 'firmware'},
 'Node_ID-order': {'dtype': 'array', 'shape': [], 'source': 'order'}}


## SECoP_BH_M_0
description and metadata

In [5]:
pprint(await bhm0.describe())
pprint(await bhm0.describe_configuration())

{'Node_ID-SECoP_BH_M_0-value': {'SECOP_datainfo': {'type': 'double',
                                                   'unit': 'mln/min'},
                                'SECoP_dtype': 'double',
                                'description': 'actual flow',
                                'dtype': 'number',
                                'readonly': True,
                                'shape': [],
                                'source': '192.168.15.4:2201:Node_ID:SECoP_BH_M_0:value',
                                'unit': 'mln/min'}}
{'Node_ID-SECoP_BH_M_0-description': {'dtype': 'string',
                                      'shape': [],
                                      'source': 'description'},
 'Node_ID-SECoP_BH_M_0-interface_classes': {'dtype': 'array',
                                            'shape': [],
                                            'source': 'interface_classes'},
 'Node_ID-SECoP_BH_M_0-order': {'dtype': 'array',
                                'sha

# SECoP_BH_M_1
description and metadata

In [ ]:
pprint(await bhm1.describe())
pprint(await bhm1.describe_configuration())

## Simple Move both in paralell


In [3]:


bhm0_target = 6.0 # ml/min
bhm1_target = 0.7 # Bar


# Simple Move both divables in Paralell
RE(bps.mov(bhm0,bhm0_target,bhm1,bhm1_target))





Executing plan <generator object mv at 0x7f2befb9ff40>
Change state on <bluesky.run_engine.RunEngine object at 0x7f2c2c5f63e0> from 'idle' -> 'running'
set(<secop_ophyd.SECoPDevices.SECoPMoveableDevice object at 0x7f2c30778760>, *(6.0,) **{'group': '61ed123d-d5d2-4d59-927b-6e5cb978bf69'}, run=None)
set(<secop_ophyd.SECoPDevices.SECoPMoveableDevice object at 0x7f2befb25690>, *(0.7,) **{'group': '61ed123d-d5d2-4d59-927b-6e5cb978bf69'}, run=None)
wait(None, *() **{'group': '61ed123d-d5d2-4d59-927b-6e5cb978bf69'}, run=None)
The object <secop_ophyd.SECoPDevices.SECoPMoveableDevice object at 0x7f2c30778760> reports set is done with status True
The object <secop_ophyd.SECoPDevices.SECoPMoveableDevice object at 0x7f2befb25690> reports set is done with status True
Change state on <bluesky.run_engine.RunEngine object at 0x7f2c2c5f63e0> from 'running' -> 'idle'
Cleaned up from plan <generator object mv at 0x7f2befb9ff40>


()

## Read one module while driving the other in steps

In [4]:

# Reads SECoP_BH_M_0 ten times while bhm1 moves from 200 to 250 
RE(scan([bhm1],bhm0,5,20,num=10))



Executing plan <generator object scan at 0x7f2befb9f680>
Change state on <bluesky.run_engine.RunEngine object at 0x7f2c2c5f63e0> from 'idle' -> 'running'
stage(<secop_ophyd.SECoPDevices.SECoP_Node_Device object at 0x7f2c30778190>, *() **{'group': 'c588df8c-5705-4585-aa28-c2e3da829194'}, run=None)
wait(None, *() **{'group': 'c588df8c-5705-4585-aa28-c2e3da829194'}, run=None)
The object <secop_ophyd.SECoPDevices.SECoP_Node_Device object at 0x7f2c30778190> reports 'stage' is done with status True.
open_run(None, *() **{'detectors': ['Node_ID-SECoP_BH_M_1'], 'motors': ('Node_ID-SECoP_BH_M_0',), 'num_points': 10, 'num_intervals': 9, 'plan_args': {'detectors': ['<secop_ophyd.SECoPDevices.SECoPMoveableDevice object at 0x7f2befb25690>'], 'num': 10, 'args': ['<secop_ophyd.SECoPDevices.SECoPMoveableDevice object at 0x7f2c30778760>', 5, 20], 'per_step': 'None'}, 'plan_name': 'scan', 'hints': {}, 'plan_pattern': 'inner_product', 'plan_pattern_module': 'bluesky.plan_patterns', 'plan_pattern_args': {

('82398454-1459-49b3-903f-d4a506323b7b',)

In [ ]:
run=db[-1]
data = run.primary.read()
data[bhm1.value.name].plot()
